# Kinematic generator
This notebook converts a Denavit Hartenberg table to its kinematic symbolic calculation and its jacobian.

In [1]:
from cmath import pi
import numpy as np
import sympy
# from sympy.abc import x,y,z,a,d
from kinematicBuilder import *

# import sys
# sys.set_int_max_str_digits(0)

# Examples

## Human arm model

### Human Arm (8 DOF)
Human arm modeled as 3 rotational joints for the shoulder (abduction, flexion, rotation), 2 joints for the elbow (flexion, pronation), 2 joints for the wrist and one of the hand

In [2]:
arm  = 10
farm = 5
palm = 1
fing = 1.2

T_shz = DenavitRow(0    , 0    , 0    , -pi/2, Joint(sympy.Symbol('sh_z'),JointType.ROTATIONAL,upper_limit=165*pi/180, lower_limit=-pi/2))
T_shy = DenavitRow(pi/2 , 0    , 0    , pi/2,  Joint(sympy.Symbol('sh_y'),JointType.ROTATIONAL,upper_limit=pi/2,       lower_limit=-pi/2))
T_shx = DenavitRow(-pi/2, -arm , 0    , pi/2,  Joint(sympy.Symbol('sh_x'),JointType.ROTATIONAL,upper_limit=pi/2,       lower_limit=-pi/2))
T_elz = DenavitRow(0    , 0    , 0    , -pi/2, Joint(sympy.Symbol('el_z'),JointType.ROTATIONAL,upper_limit=165*pi/180, lower_limit=0))
T_elx = DenavitRow(0    , -farm, 0    , pi/2,  Joint(sympy.Symbol('el_x'),JointType.ROTATIONAL,upper_limit=pi/6,       lower_limit=-110*pi/180))
T_wrz = DenavitRow(pi/2 , 0    , 0    , -pi/2, Joint(sympy.Symbol('wr_z'),JointType.ROTATIONAL,upper_limit=10*pi/180,  lower_limit=-pi/6))
T_wry = DenavitRow(0    , 0    , -palm, 0,     Joint(sympy.Symbol('wr_y'),JointType.ROTATIONAL,upper_limit=pi/3,       lower_limit=-pi/2))
T_hdy = DenavitRow(0    , 0    , -fing, 0,     Joint(sympy.Symbol('hd_y'),JointType.ROTATIONAL,upper_limit=5*pi/180,   lower_limit=-pi/2))

In [3]:
T_arm8 = DenavitDK((T_shz,T_shy,T_shx,T_elz,T_elx,T_wrz,T_wry,T_hdy),"humanArm8")
T_arm8.genURDF()

### Human Arm (5 DOF)
Human arm modeled as 3 rotational joints for the shoulder (abduction, flexion, rotation), 2 joints for the elbow (flexion, pronation) and a fixed thumb

In [4]:
thumb = 0.8
T_thb = DenavitRow(0, 0, thumb, 0)

T_arm5 = DenavitDK((T_shz,T_shy,T_shx,T_elz,T_elx,T_thb),"humanArm5")
T_arm5.genURDF()

## 3 DOF Cartesian 
A 3 DOF Cartesian robot with 3 prismatic joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [5]:
T1 = DenavitRow(pi/2,   0, 0, pi/2,  Joint(sympy.Symbol('q_1'),JointType.PRISMATIC))
T2 = DenavitRow(pi/2,   0, 0, -pi/2, Joint(sympy.Symbol('q_2'),JointType.PRISMATIC))
T3 = DenavitRow(0,      0, 0, 0,     Joint(sympy.Symbol('q_3'),JointType.PRISMATIC))

T_cartesian = DenavitDK((T1,T2,T3),"Cartesian")

In [6]:
T_cartesian.inversePositionEval([0,0,0],[1,2,3])

0.0
Solution with error 0.0 in 1 iterations


array([ 3.,  1., -2.])

In [7]:
# T_cartesian.genURDF()

In [8]:
cartesian = T_cartesian.jacobian * T_cartesian.jointsSym
cartesian

Matrix([
[ q_2],
[-q_3],
[ q_1],
[   0],
[   0],
[   0]])

In [9]:
joint_values = (0, 0, 0)
T_cartesian.eval(joint_values)

array([[ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 1,  0,  0,  0],
       [ 0,  0,  0,  1]])

## 3 DOF Articulated Robot
A 3 DOF Articulated robot with 3 rotational joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [10]:
T1 = DenavitRow(0,  1.5, 0,   pi/2, Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,  0,  0.5,  0,    Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))

T_articular3 = DenavitDK((T1,T2,T3))

In [11]:
# A=T_articular3.directTransformSym
# U,S,V = A.singular_value_decomposition() # Memory error

In [12]:
joint_values = (0, 0, 0)
T_articular3.eval(joint_values)

array([[ 1. , -0. ,  0. ,  1.5],
       [ 0. , -0. , -1. ,  0. ],
       [ 0. ,  1. ,  0. ,  1.5],
       [ 0. ,  0. ,  0. ,  1. ]])